![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🧠 Section 3: Memory Architecture - From Stateless RAG to Stateful Conversations

**⏱️ Estimated Time:** 45-60 minutes

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** why memory is essential for context engineering
2. **Implement** working memory for conversation continuity
3. **Use** long-term memory for persistent user knowledge
4. **Integrate** memory with your Section 2 RAG system
5. **Build** a complete memory-enhanced course advisor

---

## 🔗 Recap

### **Section 1: The Four Context Types**

Recall the four context types from Section 1:

1. **System Context** (Static) - Role, instructions, guidelines
2. **User Context** (Dynamic, User-Specific) - Profile, preferences, goals
3. **Conversation Context** (Dynamic, Session-Specific) - **← Memory enables this!**
4. **Retrieved Context** (Dynamic, Query-Specific) - RAG results

### **Section 2: Stateless RAG**

Your Section 2 RAG system was **stateless**:

```python
async def rag_query(query, student_profile):
    # 1. Search courses (Retrieved Context)
    courses = await course_manager.search_courses(query)

    # 2. Assemble context (System + User + Retrieved)
    context = assemble_context(system_prompt, student_profile, courses)

    # 3. Generate response
    response = llm.invoke(context)

    # ❌ No conversation history stored
    # ❌ Each query is independent
    # ❌ Can't reference previous messages
```

**The Problem:** Every query starts from scratch. No conversation continuity.

---

## 🚨 Why Agents Need Memory: The Grounding Problem

Before diving into implementation, let's understand the fundamental problem that memory solves.

**Grounding** means understanding what users are referring to. Natural conversation is full of references:

### **Without Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers supervised learning..."

User: "What are its prerequisites?"
Agent: ❌ "What does 'it' refer to? Please specify which course."

User: "The course we just discussed!"
Agent: ❌ "I don't have access to previous messages. Which course?"
```

**This is a terrible user experience.**

### Types of References That Need Grounding

**Pronouns:**
- "it", "that course", "those", "this one"
- "he", "she", "they" (referring to people)

**Descriptions:**
- "the easy one", "the online course"
- "my advisor", "that professor"

**Implicit context:**
- "Can I take it?" → Take what?
- "When does it start?" → What starts?

**Temporal references:**
- "you mentioned", "earlier", "last time"

### **With Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers..."
[Stores: User asked about CS401]

User: "What are its prerequisites?"
Agent: [Checks memory: "its" = CS401]
Agent: ✅ "CS401 requires CS201 and MATH301"

User: "Can I take it?"
Agent: [Checks memory: "it" = CS401, checks student transcript]
Agent: ✅ "You've completed CS201 but still need MATH301"
```

**Now the conversation flows naturally!**

---

## 🧠 Two Types of Memory

### **1. Working Memory (Session-Scoped)**

 - **What:** Conversation messages from the current session
 - **Purpose:** Reference resolution, conversation continuity
 - **Lifetime:** Session duration (24 hours TTL by default)

**Example:**
```
Session: session_123
Messages:
  1. User: "Tell me about CS401"
  2. Agent: "CS401 is Machine Learning..."
  3. User: "What are its prerequisites?"
  4. Agent: "CS401 requires CS201 and MATH301"
```

### **2. Long-term Memory (Cross-Session)**

 - **What:** Persistent facts, preferences, goals
 - **Purpose:** Personalization across sessions and applications
 - **Lifetime:** Permanent (until explicitly deleted)

**Example:**
```
User: student_sarah
Memories:
  - "Prefers online courses over in-person"
  - "Major: Computer Science, focus on AI/ML"
  - "Goal: Graduate Spring 2026"
  - "Completed: CS101, CS201, MATH301"
```

### **Comparison: Working vs. Long-term Memory**

| Working Memory | Long-term Memory |
|----------------|------------------|
| **Session-scoped** | **User-scoped** |
| Current conversation | Important facts |
| TTL-based (expires) | Persistent |
| Full message history | Extracted knowledge |
| Loaded/saved each turn | Searched when needed |

---

## 📚 Part 1: Working Memory Fundamentals

### **What is Working Memory?**

Working memory stores **conversation messages** for the current session. It enables:

- ✅ **Reference resolution** - "it", "that course", "the one you mentioned"
- ✅ **Context continuity** - Each message builds on previous messages
- ✅ **Natural conversations** - Users don't repeat themselves

### **How It Works:**

```
Turn 1: Load working memory (empty) → Process query → Save messages
Turn 2: Load working memory (1 exchange) → Process query → Save messages
Turn 3: Load working memory (2 exchanges) → Process query → Save messages
```

Each turn has access to all previous messages in the session.

---

## 🧪 Hands-On: Working Memory in Action

Let's simulate a multi-turn conversation with working memory.


In [6]:
# Working Memory Demo
async def working_memory_demo():
    """Demonstrate working memory for conversation continuity"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"
    session_id = f"session_{student_id}_demo"

    print("=" * 80)
    print("🧪 WORKING MEMORY DEMO: Multi-Turn Conversation")
    print("=" * 80)

    # Turn 1: First query
    print("\n📍 TURN 1: User asks about a course")
    print("-" * 80)

    user_query_1 = "Tell me about CS401"

    # Load working memory (empty for first turn)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   Messages in working memory: {len(working_memory.messages)}")
    print(f"   User: {user_query_1}")

    # Search for course
    courses = await course_manager.search_courses(user_query_1, limit=1)

    # Generate response (simplified - no full RAG for demo)
    if courses:
        course = courses[0]
        response_1 = f"{course.course_code}: {course.title}. {course.description[:100]}..."
    else:
        response_1 = "I couldn't find that course."

    print(f"   Agent: {response_1}")

    # Save to working memory
    working_memory.messages.extend([
        MemoryMessage(role="user", content=user_query_1),
        MemoryMessage(role="assistant", content=response_1)
    ])

    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   ✅ Saved to working memory")

    # Turn 2: Follow-up with pronoun reference
    print("\n📍 TURN 2: User uses pronoun reference ('its')")
    print("-" * 80)

    user_query_2 = "What are its prerequisites?"

    # Load working memory (now has 1 exchange)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   Messages in working memory: {len(working_memory.messages)}")
    print(f"   User: {user_query_2}")

    # Build context with conversation history
    messages = [
        SystemMessage(content="You are a helpful course advisor. Use conversation history to resolve references like 'it', 'that course', etc.")
    ]

    # Add conversation history from working memory
    for msg in working_memory.messages:
        if msg.role == "user":
            messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            messages.append(AIMessage(content=msg.content))

    # Add current query
    messages.append(HumanMessage(content=user_query_2))

    # Generate response (LLM can now resolve "its" using conversation history)
    response_2 = llm.invoke(messages).content

    print(f"   Agent: {response_2}")

    # Save to working memory
    working_memory.messages.extend([
        MemoryMessage(role="user", content=user_query_2),
        MemoryMessage(role="assistant", content=response_2)
    ])

    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   ✅ Saved to working memory")

    # Turn 3: Another follow-up
    print("\n📍 TURN 3: User asks another follow-up")
    print("-" * 80)

    user_query_3 = "Can I take it next semester?"

    # Load working memory (now has 2 exchanges)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   Messages in working memory: {len(working_memory.messages)}")
    print(f"   User: {user_query_3}")

    # Build context with full conversation history
    messages = [
        SystemMessage(content="You are a helpful course advisor. Use conversation history to resolve references.")
    ]

    for msg in working_memory.messages:
        if msg.role == "user":
            messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            messages.append(AIMessage(content=msg.content))

    messages.append(HumanMessage(content=user_query_3))

    response_3 = llm.invoke(messages).content

    print(f"   Agent: {response_3}")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Working memory enabled natural conversation flow!")
    print("=" * 80)

# Run the demo
await working_memory_demo()


### 🎯 What Just Happened?

**Turn 1:** User asks about CS401
- Working memory: **empty**
- Agent responds with course info
- Saves: User query + Agent response

**Turn 2:** User asks "What are **its** prerequisites?"
- Working memory: **1 exchange** (Turn 1)
- LLM resolves "its" → CS401 (from conversation history)
- Agent answers correctly
- Saves: Updated conversation

**Turn 3:** User asks "Can I take **it** next semester?"
- Working memory: **2 exchanges** (Turns 1-2)
- LLM resolves "it" → CS401 (from conversation history)
- Agent answers correctly

**💡 Key Insight:** Working memory enables **reference resolution** and **conversation continuity**.

---

## 📚 Three Types of Long-term Memories

Long-term memory isn't just one thing - the Agent Memory Server supports **three distinct types**, each optimized for different kinds of information:

### **1. Semantic Memory - Facts and Knowledge**

**What it stores:** Timeless facts, preferences, and knowledge that don't depend on when they were learned.

**Examples:**
- "Student prefers online courses"
- "Student's major is Computer Science"
- "Student wants to graduate in Spring 2026"
- "Student struggles with mathematics"
- "Student is interested in machine learning"

**When to use:** For information that remains true regardless of time context.

---

### **2. Episodic Memory - Events and Experiences**

**What it stores:** Time-bound events, experiences, and timeline-based information.

**Examples:**
- "Student enrolled in CS101 on 2024-09-15"
- "Student completed CS101 with grade A on 2024-12-10"
- "Student asked about machine learning courses on 2024-09-20"
- "Student expressed concerns about workload on 2024-10-27"

**When to use:** When the timing or sequence of events matters.

---

### **3. Message Memory - Context-Rich Conversations**

**What it stores:** Full conversation snippets where complete context is crucial.

**Examples:**
- Detailed career planning discussion with nuanced advice
- Professor's specific guidance about research opportunities
- Student's explanation of personal learning challenges

**When to use:** When summary would lose important nuance, tone, or context.

**⚠️ Use sparingly** - Message memories are token-expensive!

---

## 🎯 Choosing the Right Memory Type

Understanding **when** to use each memory type is crucial for effective memory management. Let's explore a decision framework.

### **Decision Framework**

#### **Use Semantic Memory for: Facts and Preferences**

**Characteristics:**
- Timeless information (not tied to specific moment)
- Likely to be referenced repeatedly
- Can be stated independently of context

**Examples:**
```python
# ✅ Good semantic memories
"Student prefers online courses"
"Student's major is Computer Science"
"Student wants to graduate in Spring 2026"
"Student struggles with mathematics"
"Student is interested in machine learning"
```

**Why semantic:**
- Facts that don't change often
- Will be useful across many sessions
- Don't need temporal context

---

#### **Use Episodic Memory for: Events and Timeline**

**Characteristics:**
- Time-bound events
- Sequence/timeline matters
- Tracking progress or history

**Examples:**
```python
# ✅ Good episodic memories
"Student enrolled in CS101 on 2024-09-15"
"Student completed CS101 on 2024-12-10"
"Student started CS201 on 2024-01-15"
"Student asked about career planning on 2024-10-20"
"Student expressed concerns about workload on 2024-10-27"
```

**Why episodic:**
- Events have specific dates
- Order of events matters (CS101 before CS201)
- Tracking student's journey over time

---

#### **Use Message Memory for: Context-Rich Conversations**

**Characteristics:**
- Full context is crucial
- Tone/emotion matters
- May need exact wording
- Complex multi-part discussions

**Examples:**
```python
# ✅ Good message memories
"Detailed career planning discussion: [full conversation]"
"Professor's specific advice about research opportunities: [full message]"
"Student's explanation of personal learning challenges: [full message]"
```

**Why message:**
- Summary would lose important nuance
- Context around the words matters
- Verbatim quote may be needed

**⚠️ Use sparingly** - Message memories are token-expensive!

---

### **Examples: Right vs. Wrong**

#### **Scenario 1: Student States Preference**

**User says:** "I prefer online courses because I work during the day."

❌ **Wrong:**
```python
# Message memory (too verbose)
memory = "Student said: 'I prefer online courses because I work during the day.'"
```

✅ **Right:**
```python
# Semantic memories (extracted facts)
memory1 = "Student prefers online courses"
memory2 = "Student works during the day"
```

**Why:** Simple facts don't need full verbatim storage.

---

#### **Scenario 2: Course Completion**

**User says:** "I just finished CS101 last week!"

❌ **Wrong:**
```python
# Semantic (loses temporal context)
memory = "Student completed CS101"
```

✅ **Right:**
```python
# Episodic (preserves timeline)
memory = "Student completed CS101 on 2024-10-20"
```

**Why:** Timeline matters for prerequisites and planning.

---

#### **Scenario 3: Complex Career Advice**

**Conversation:** 20-message discussion about career path, including professor's nuanced advice about research vs. industry, timing of applications, and specific companies to target.

❌ **Wrong:**
```python
# Semantic (loses too much)
memory = "Student discussed career planning"
```

✅ **Right:**
```python
# Message memory (preserves context)
memory = [Full conversation thread with all nuance]
```

**Why:** Details and context are critical, summary inadequate.

---

### **Quick Reference Table**

| Information Type | Memory Type | Example |
|-----------------|-------------|----------|
| Preference | Semantic | "Prefers morning classes" |
| Fact | Semantic | "Major is Computer Science" |
| Goal | Semantic | "Wants to graduate in 2026" |
| Event | Episodic | "Enrolled in CS401 on 2024-09-15" |
| Timeline | Episodic | "Completed CS101, then CS201" |
| Progress | Episodic | "Asked about ML three times" |
| Complex discussion | Message | [Full career planning conversation] |
| Nuanced advice | Message | [Professor's detailed guidance] |

### **Default Strategy: Prefer Semantic**

**When in doubt:**
1. Can you extract a simple fact? → **Semantic**
2. Is timing important? → **Episodic**
3. Is full context crucial? → **Message** (use rarely)

**Most memories should be semantic** - they're compact, searchable, and efficient.

---

## 📚 Part 2: Long-term Memory Fundamentals

### **What is Long-term Memory?**

Long-term memory stores **persistent facts, preferences, and goals** across sessions. It enables:

✅ **Personalization** - Remember user preferences across conversations
✅ **Knowledge accumulation** - Build understanding over time
✅ **Semantic search** - Find relevant memories using natural language

### **Memory Types:**

1. **Semantic** - Facts and knowledge ("Prefers online courses")
2. **Episodic** - Events and experiences ("Enrolled in CS101 on 2024-09-01")
3. **Message** - Important conversation excerpts

### **How It Works:**

```
Session 1: User shares preferences → Store in long-term memory
Session 2: User asks for recommendations → Search long-term memory → Personalized response
Session 3: User updates preferences → Update long-term memory
```

Long-term memory persists across sessions and is searchable via semantic vector search.

---

## 🧪 Hands-On: Long-term Memory in Action

Let's store and search long-term memories.


In [ ]:
# Long-term Memory Demo
async def longterm_memory_demo():
    """Demonstrate long-term memory for persistent knowledge"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"

    print("=" * 80)
    print("🧪 LONG-TERM MEMORY DEMO: Persistent Knowledge")
    print("=" * 80)

    # Step 1: Store semantic memories (facts)
    print("\n📍 STEP 1: Storing Semantic Memories (Facts)")
    print("-" * 80)

    semantic_memories = [
        "Student prefers online courses over in-person classes",
        "Student's major is Computer Science with focus on AI/ML",
        "Student wants to graduate in Spring 2026",
        "Student prefers morning classes, no classes on Fridays",
        "Student has completed CS101 and CS201",
        "Student is currently taking MATH301"
    ]

    for memory_text in semantic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=student_id,
            memory_type="semantic",
            topics=["preferences", "academic_info"]
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ Stored: {memory_text}")

    # Step 2: Store episodic memories (events)
    print("\n📍 STEP 2: Storing Episodic Memories (Events)")
    print("-" * 80)

    episodic_memories = [
        "Student enrolled in CS101 on 2024-09-01",
        "Student completed CS101 with grade A on 2024-12-15",
        "Student asked about machine learning courses on 2024-09-20"
    ]

    for memory_text in episodic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=student_id,
            memory_type="episodic",
            topics=["enrollment", "courses"]
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ Stored: {memory_text}")

    # Step 3: Search long-term memory with semantic queries
    print("\n📍 STEP 3: Searching Long-term Memory")
    print("-" * 80)

    search_queries = [
        "What does the student prefer?",
        "What courses has the student completed?",
        "What is the student's major?"
    ]

    for query in search_queries:
        print(f"\n   🔍 Query: '{query}'")
        results = await memory_client.search_long_term_memory(
            text=query,
            user_id=student_id,
            limit=3
        )

        if results.memories:
            print(f"   📚 Found {len(results.memories)} relevant memories:")
            for i, memory in enumerate(results.memories[:3], 1):
                print(f"      {i}. {memory.text}")
        else:
            print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Long-term memory enables persistent knowledge!")
    print("=" * 80)

# Run the demo
await longterm_memory_demo()


### 🎯 What Just Happened?

**Step 1: Stored Semantic Memories**
- Created 6 semantic memories (facts about student)
- Tagged with topics for organization
- Stored in vector database for semantic search

**Step 2: Stored Episodic Memories**
- Created 3 episodic memories (time-bound events)
- Captures timeline of student's academic journey
- Also searchable via semantic search

**Step 3: Searched Long-term Memory**
- Used natural language queries
- Semantic search found relevant memories
- No exact keyword matching needed

**💡 Key Insight:** Long-term memory enables **personalization** and **knowledge accumulation** across sessions.

---

## 🏗️ Memory Architecture

We'll use **Redis Agent Memory Server** - a production-ready dual-memory system:

**Working Memory:**
- Session-scoped conversation context
- Automatic extraction to long-term storage
- TTL-based expiration

**Long-term Memory:**
- Vector-indexed for semantic search
- Automatic deduplication
- Three types: semantic (facts), episodic (events), message

### **How Automatic Deduplication Works**

The Agent Memory Server prevents duplicate memories using two strategies:

1. **Hash-based Deduplication:** Exact duplicates are rejected
   - Same text = same hash = rejected
   - Prevents storing identical memories multiple times

2. **Semantic Deduplication:** Similar memories are merged
   - "Student prefers online courses" ≈ "Student likes taking classes online"
   - Vector similarity detects semantic overlap
   - Keeps memory storage efficient

**Result:** Your memory store stays clean and efficient without manual cleanup!

**Why Agent Memory Server?**
- Production-ready (handles thousands of users)
- Redis-backed (fast, scalable)
- Automatic memory management (extraction, deduplication)
- Semantic search built-in

---

## 📦 Setup

### **What We're Importing:**

- **Section 2 components** - `redis_config`, `CourseManager`, models
- **Agent Memory Server client** - `MemoryAPIClient` for memory operations
- **LangChain** - `ChatOpenAI` for LLM interaction

### **Why:**

- Build on Section 2's RAG foundation
- Add memory capabilities without rewriting everything
- Use production-ready memory infrastructure


In [ ]:
# Setup: Import components
import os
import sys
import asyncio
from typing import List, Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

# Load environment
load_dotenv()
sys.path.append('../../reference-agent')

# Import Section 2 components
from redis_context_course.redis_config import redis_config
from redis_context_course.course_manager import CourseManager
from redis_context_course.models import (
    Course, StudentProfile, DifficultyLevel,
    CourseFormat, Semester
)

# Import LangChain
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import WorkingMemory, MemoryMessage, ClientMemoryRecord
    MEMORY_SERVER_AVAILABLE = True
    print("✅ Agent Memory Server client available")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("⚠️  Agent Memory Server not available")
    print("📝 Install with: pip install agent-memory-client")
    print("🚀 Start server: See reference-agent/README.md")

# Verify environment
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found. Please set in .env file.")
else:
    print("✅ OPENAI_API_KEY found")

print(f"\n🔧 Environment Setup:")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"   REDIS_URL: {os.getenv('REDIS_URL', 'redis://localhost:6379')}")
print(f"   AGENT_MEMORY_URL: {os.getenv('AGENT_MEMORY_URL', 'http://localhost:8088')}")
print(f"   Memory Server: {'✓ Available' if MEMORY_SERVER_AVAILABLE else '✗ Not available'}")


### 🎯 What We Just Did

**Successfully Imported:**
- ✅ **Section 2 RAG components** - `redis_config`, `CourseManager`, models
- ✅ **Agent Memory Server client** - Production-ready memory system
- ✅ **Environment verified** - OpenAI API key, Redis, Memory Server

**Why This Matters:**
- We're **building on Section 2's foundation** (not starting from scratch)
- **Agent Memory Server** provides scalable, persistent memory
- **Same Redis University domain** for consistency

---

## 🔧 Initialize Components


In [ ]:
# Initialize components
course_manager = CourseManager()
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Initialize Memory Client
if MEMORY_SERVER_AVAILABLE:
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8088"),
        default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    print("🧠 Memory Client Initialized")
    print(f"   Base URL: {config.base_url}")
    print(f"   Namespace: {config.default_namespace}")
else:
    memory_client = None
    print("⚠️  Running without Memory Server (limited functionality)")

# Create a sample student profile (reusing Section 2 pattern)
sarah = StudentProfile(
    name="Sarah Chen",
    email="sarah.chen@university.edu",
    major="Computer Science",
    year=2,
    interests=["machine learning", "data science", "algorithms"],
    completed_courses=["CS101", "CS201"],
    current_courses=["MATH301"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE
)

print(f"\n👤 Student Profile: {sarah.name}")
print(f"   Major: {sarah.major}")
print(f"   Interests: {', '.join(sarah.interests)}")


### 💡 Key Insight

We're reusing:
- ✅ **Same `CourseManager`** from Section 2
- ✅ **Same `StudentProfile`** model
- ✅ **Same Redis configuration**

We're adding:
- ✨ **Memory Client** for conversation history
- ✨ **Working Memory** for session context
- ✨ **Long-term Memory** for persistent knowledge

---

## 🏷️ Advanced: Topics and Filtering

Topics help organize and filter memories. Let's explore how to use them effectively.


In [ ]:
# Topics and Filtering Demo
async def topics_filtering_demo():
    """Demonstrate topics and filtering for memory organization"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"

    print("=" * 80)
    print("🏷️  TOPICS AND FILTERING DEMO")
    print("=" * 80)

    # Store memories with specific topics
    print("\n📍 Storing Memories with Topics")
    print("-" * 80)

    memories_with_topics = [
        ("Student prefers online courses", ["preferences", "course_format"]),
        ("Student's major is Computer Science", ["academic_info", "major"]),
        ("Student wants to graduate in Spring 2026", ["goals", "graduation"]),
        ("Student prefers morning classes", ["preferences", "schedule"]),
    ]

    for memory_text, topics in memories_with_topics:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=student_id,
            memory_type="semantic",
            topics=topics
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")
        print(f"      Topics: {', '.join(topics)}")

    # Filter by memory type
    print("\n📍 Filtering by Memory Type: Semantic")
    print("-" * 80)

    from agent_memory_client.models import MemoryType

    results = await memory_client.search_long_term_memory(
        text="",  # Empty query returns all
        user_id=student_id,
        memory_type=MemoryType(eq="semantic"),
        limit=10
    )

    print(f"   Found {len(results.memories)} semantic memories:")
    for i, memory in enumerate(results.memories[:5], 1):
        topics_str = ', '.join(memory.topics) if memory.topics else 'none'
        print(f"   {i}. {memory.text}")
        print(f"      Topics: {topics_str}")

    print("\n" + "=" * 80)
    print("✅ Topics enable organized, filterable memory management!")
    print("=" * 80)

# Run the demo
await topics_filtering_demo()


### 🎯 Why Topics Matter

**Organization:**
- Group related memories together
- Easy to find memories by category

**Filtering:**
- Search within specific topics
- Filter by memory type (semantic, episodic, message)

**Best Practices:**
- Use consistent topic names
- Keep topics broad enough to be useful
- Common topics: `preferences`, `academic_info`, `goals`, `schedule`, `courses`

---

## 🔄 Cross-Session Memory Persistence

Let's verify that memories persist across sessions.


In [ ]:
# Cross-Session Demo
async def cross_session_demo():
    """Demonstrate memory persistence across sessions"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"

    print("=" * 80)
    print("🔄 CROSS-SESSION MEMORY PERSISTENCE DEMO")
    print("=" * 80)

    # Simulate Session 1: Store memories
    print("\n📍 SESSION 1: Storing Memories")
    print("-" * 80)

    memory_record = ClientMemoryRecord(
        text="Student is interested in machine learning and AI",
        user_id=student_id,
        memory_type="semantic",
        topics=["interests", "AI"]
    )
    await memory_client.create_long_term_memory([memory_record])
    print("   ✅ Stored: Student is interested in machine learning and AI")

    # Simulate Session 2: Create new client (new session)
    print("\n📍 SESSION 2: New Session, Same Student")
    print("-" * 80)

    # Create a new memory client (simulating a new session)
    new_session_config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
        default_namespace="redis_university"
    )
    new_session_client = MemoryAPIClient(config=new_session_config)

    print("   🔄 New session started for the same student")

    # Search for memories from the new session
    print("\n   🔍 Searching: 'What are the student's interests?'")
    results = await new_session_client.search_long_term_memory(
        text="What are the student's interests?",
        user_id=student_id,
        limit=3
    )

    if results.memories:
        print(f"\n   ✅ Memories accessible from new session:")
        for i, memory in enumerate(results.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ Long-term memories persist across sessions!")
    print("=" * 80)

# Run the demo
await cross_session_demo()


### 🎯 Cross-Session Persistence

**What We Demonstrated:**
- **Session 1:** Stored memories about student interests
- **Session 2:** Created new client (simulating new session)
- **Result:** Memories from Session 1 are accessible in Session 2

**Why This Matters:**
- Users don't have to repeat themselves
- Personalization works across days, weeks, months
- Knowledge accumulates over time

**Contrast with Working Memory:**
- Working memory: Session-scoped (expires after 24 hours)
- Long-term memory: User-scoped (persists indefinitely)

---

## 🔗 What's Next: Memory-Enhanced RAG and Agents

You've learned the fundamentals of memory architecture! Now it's time to put it all together.

### **Next Notebook: `02_memory_enhanced_rag_and_agents.ipynb`**

In the next notebook, you'll:

1. **Build** a complete memory-enhanced RAG system
   - Integrate working memory + long-term memory + RAG
   - Combine all four context types
   - Show clear before/after comparisons

2. **Convert** to LangGraph agent (Part 2, separate notebook)
   - Add state management
   - Improve control flow
   - Prepare for Section 4 (tools and advanced capabilities)

**Why Continue?**
- See memory in action with real conversations
- Learn how to build production-ready agents
- Prepare for Section 4 (adding tools like enrollment, scheduling)

**📚 Continue to:** `02_memory_enhanced_rag_and_agents.ipynb`

## ⏰ Memory Lifecycle & Persistence

Understanding how long memories last and when they expire is crucial for building reliable systems.

### **Working Memory TTL (Time-To-Live)**

**Default TTL:** 24 hours

**What this means:**
- Working memory (conversation history) expires 24 hours after last activity
- After expiration, conversation context is lost
- Long-term memories extracted from the conversation persist

**Timeline Example:**

```
Day 1, 10:00 AM - Session starts
Day 1, 10:25 AM - Session ends
    ↓
[24 hours later]
    ↓
Day 2, 10:25 AM - Working memory still available ✅
Day 2, 10:26 AM - Working memory expires ❌
```

### **Long-term Memory Persistence**

**Lifetime:** Indefinite (until manually deleted)

**What this means:**
- Long-term memories never expire automatically
- Accessible across all sessions, forever
- Must be explicitly deleted if no longer needed

### **Why This Design?**

**Working Memory (Short-lived):**
- Conversations are temporary
- Most context is only relevant during the session
- Automatic cleanup prevents storage bloat
- Privacy: Old conversations don't linger

**Long-term Memory (Persistent):**
- Important facts should persist
- User preferences don't expire
- Knowledge accumulates over time
- Enables true personalization

### **Important Implications**

**1. Extract Before Expiration**

If something important is said in conversation, it must be extracted to long-term memory before the 24-hour TTL expires.

**Good news:** Agent Memory Server does this automatically!

**2. Long-term Memories are Permanent**

Once stored, long-term memories persist indefinitely. Be thoughtful about what you store.

**3. Cross-Session Behavior**

```
Session 1 (Day 1):
- User: "I'm interested in machine learning"
- Working memory: Stores conversation
- Long-term memory: Extracts "Student interested in machine learning"

[30 hours later - Working memory expired]

Session 2 (Day 3):
- Working memory from Session 1: EXPIRED ❌
- Long-term memory: Still available ✅
- Agent retrieves: "Student interested in machine learning"
- Agent makes relevant recommendations ✅
```

### **Practical Multi-Day Conversation Example**


In [ ]:
# Multi-Day Conversation Simulation
async def multi_day_simulation():
    """Simulate conversations across multiple days"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"

    print("=" * 80)
    print("⏰ MULTI-DAY CONVERSATION SIMULATION")
    print("=" * 80)

    # Day 1: Initial conversation
    print("\n📅 DAY 1: Initial Conversation")
    print("-" * 80)

    session_1 = f"session_{student_id}_day1"

    # Store a fact in long-term memory
    memory_record = ClientMemoryRecord(
        text="Student is preparing for a career in AI research",
        user_id=student_id,
        memory_type="semantic",
        topics=["career", "goals"]
    )
    await memory_client.create_long_term_memory([memory_record])
    print("   ✅ Stored in long-term memory: Career goal (AI research)")

    # Simulate working memory (would normally be conversation)
    print("   💬 Working memory: Active for session_day1")
    print("   ⏰ TTL: 24 hours from now")

    # Day 3: New conversation (working memory expired)
    print("\n📅 DAY 3: New Conversation (48 hours later)")
    print("-" * 80)

    session_2 = f"session_{student_id}_day3"

    print("   ❌ Working memory from Day 1: EXPIRED")
    print("   ✅ Long-term memory: Still available")

    # Search long-term memory
    results = await memory_client.search_long_term_memory(
        text="What are the student's career goals?",
        user_id=student_id,
        limit=3
    )

    if results.memories:
        print("\n   🔍 Retrieved from long-term memory:")
        for memory in results.memories[:3]:
            print(f"      • {memory.text}")
        print("\n   ✅ Agent can still personalize recommendations!")

    print("\n" + "=" * 80)
    print("✅ Long-term memories persist, working memory expires")
    print("=" * 80)

# Run the simulation
await multi_day_simulation()


### 🎯 Memory Lifecycle Best Practices

**1. Trust Automatic Extraction**
- Agent Memory Server automatically extracts important facts
- Don't manually store everything in long-term memory
- Let the system decide what's important

**2. Use Appropriate Memory Types**
- Working memory: Current conversation only
- Long-term memory: Facts that should persist

**3. Monitor Memory Growth**
- Long-term memories accumulate over time
- Implement cleanup for outdated information
- Consider archiving old memories

**4. Plan for Expiration**
- Working memory expires after 24 hours
- Important context must be in long-term memory
- Don't rely on working memory for cross-session data

**5. Test Cross-Session Behavior**
- Verify long-term memories are accessible
- Ensure personalization works after TTL expiration
- Test with realistic time gaps

---

## 🎓 Key Takeaways

### **1. Memory Solves the Grounding Problem**

Without memory, agents can't resolve references:
- ❌ "What are **its** prerequisites?" → Agent doesn't know what "its" refers to
- ✅ With working memory → Agent resolves "its" from conversation history

### **2. Two Types of Memory Serve Different Purposes**

**Working Memory (Session-Scoped):**
- Conversation messages from current session
- Enables reference resolution and conversation continuity
- TTL-based (expires after session ends)

**Long-term Memory (Cross-Session):**
- Persistent facts, preferences, goals
- Enables personalization across sessions
- Searchable via semantic vector search

### **3. Memory Completes the Four Context Types**

From Section 1, we learned about four context types. Memory enables two of them:

1. **System Context** (Static) - ✅ Section 2
2. **User Context** (Dynamic, User-Specific) - ✅ Section 2 + Long-term Memory
3. **Conversation Context** (Dynamic, Session-Specific) - ✨ **Working Memory**
4. **Retrieved Context** (Dynamic, Query-Specific) - ✅ Section 2 RAG

### **4. Memory + RAG = Complete Context Engineering**

The integration pattern:
```
1. Load working memory (conversation history)
2. Search long-term memory (user facts)
3. RAG search (relevant documents)
4. Assemble all context types
5. Generate response
6. Save working memory (updated conversation)
```

This gives us **stateful, personalized, context-aware conversations**.

### **5. Agent Memory Server is Production-Ready**

Why use Agent Memory Server instead of simple in-memory storage:
- ✅ **Scalable** - Redis-backed, handles thousands of users
- ✅ **Automatic** - Extracts important facts to long-term storage
- ✅ **Semantic search** - Vector-indexed memory retrieval
- ✅ **Deduplication** - Prevents redundant memories
- ✅ **TTL management** - Automatic expiration of old sessions

### **6. LangChain is Sufficient for Memory + RAG**

We didn't need LangGraph for this section because:
- Simple linear flow (load → search → generate → save)
- No conditional branching or complex state management
- No tool calling required

**LangGraph becomes necessary in Section 4** when we add tools and multi-step workflows.

### **7. Memory Management Best Practices**

**Choose the Right Memory Type:**
- **Semantic** for facts and preferences (most common)
- **Episodic** for time-bound events and timeline
- **Message** for context-rich conversations (use sparingly)

**Understand Memory Lifecycle:**
- **Working memory:** 24-hour TTL, session-scoped
- **Long-term memory:** Indefinite persistence, user-scoped
- **Automatic extraction:** Trust the system to extract important facts

**Benefits of Proper Memory Management:**
- ✅ **Natural conversations** - Users don't repeat themselves
- ✅ **Cross-session personalization** - Knowledge persists over time
- ✅ **Efficient storage** - Automatic deduplication prevents bloat
- ✅ **Semantic search** - Find relevant memories without exact keywords
- ✅ **Scalable** - Redis-backed, production-ready architecture

**Key Principle:** Memory transforms stateless RAG into stateful, personalized, context-aware conversations.

---

## 🚀 What's Next?

### **Next Notebook: Memory-Enhanced RAG and Agents**

**📚 Continue to: `02_memory_enhanced_rag_and_agents.ipynb`**

In the next notebook, you'll:

1. **Build** a complete memory-enhanced RAG system
   - Integrate working memory + long-term memory + RAG
   - Combine all four context types
   - Show clear before/after comparisons

2. **Convert** to LangGraph agent (Part 2, separate notebook)
   - Add state management
   - Improve control flow
   - Prepare for Section 4 (tools and advanced capabilities)

### **Then: Section 4 - Tools and Advanced Agents**

After completing the next notebook, you'll be ready for Section 4:

**Tools You'll Add:**
- `search_courses` - Semantic search
- `get_course_details` - Fetch specific course information
- `check_prerequisites` - Verify student eligibility
- `enroll_course` - Register student for a course
- `store_memory` - Explicitly save important facts

**The Complete Learning Path:**

```
Section 1: Context Engineering Fundamentals
    ↓
Section 2: RAG (Retrieved Context)
    ↓
Section 3 (Notebook 1): Memory Fundamentals ← You are here
    ↓
Section 3 (Notebook 2): Memory-Enhanced RAG and Agents
    ↓
Section 4: Tools + Agents (Complete Agentic System)
```

---

## 💪 Practice Exercises

### **Exercise 1: Cross-Session Personalization**

Modify the `memory_enhanced_rag_query` function to:
1. Store user preferences in long-term memory when mentioned
2. Use those preferences in future sessions
3. Test with two different sessions for the same student

**Hint:** Look for phrases like "I prefer...", "I like...", "I want..." and store them as semantic memories.

### **Exercise 2: Memory-Aware Filtering**

Enhance the RAG search to use long-term memories as filters:
1. Search long-term memory for preferences (format, difficulty, schedule)
2. Apply those preferences as filters to `course_manager.search_courses()`
3. Compare results with and without memory-aware filtering

**Hint:** Use the `filters` parameter in `course_manager.search_courses()`.

### **Exercise 3: Conversation Summarization**

Implement a function that summarizes long conversations:
1. When working memory exceeds 10 messages, summarize the conversation
2. Store the summary in long-term memory
3. Clear old messages from working memory (keep only recent 4)
4. Test that reference resolution still works with summarized history

**Hint:** Use the LLM to generate summaries, then store as semantic memories.

### **Exercise 4: Multi-User Memory Management**

Create a simple CLI that:
1. Supports multiple students (different user IDs)
2. Maintains separate working memory per session
3. Maintains separate long-term memory per user
4. Demonstrates cross-session continuity for each user

**Hint:** Use different `session_id` and `user_id` for each student.

### **Exercise 5: Memory Search Quality**

Experiment with long-term memory search:
1. Store 20+ diverse memories for a student
2. Try different search queries
3. Analyze which memories are retrieved
4. Adjust memory text to improve search relevance

**Hint:** More specific memory text leads to better semantic search results.

---

## 📝 Summary

### **What You Learned:**

1. **The Grounding Problem** - Why agents need memory to resolve references
2. **Working Memory** - Session-scoped conversation history for continuity
3. **Long-term Memory** - Cross-session persistent knowledge for personalization
4. **Memory Integration** - Combining memory with Section 2's RAG system
5. **Complete Context Engineering** - All four context types working together
6. **Production Architecture** - Using Agent Memory Server for scalable memory

### **What You Built:**

- ✅ Working memory demo (multi-turn conversations)
- ✅ Long-term memory demo (persistent knowledge)
- ✅ Complete memory-enhanced RAG system
- ✅ Integration of all four context types

### **Key Functions:**

- `memory_enhanced_rag_query()` - Complete memory + RAG pipeline
- `working_memory_demo()` - Demonstrates conversation continuity
- `longterm_memory_demo()` - Demonstrates persistent knowledge
- `complete_demo()` - End-to-end multi-turn conversation

### **Architecture Pattern:**

```
User Query
    ↓
Load Working Memory (conversation history)
    ↓
Search Long-term Memory (user facts)
    ↓
RAG Search (relevant courses)
    ↓
Assemble Context (System + User + Conversation + Retrieved)
    ↓
Generate Response
    ↓
Save Working Memory (updated conversation)
```

### **From Section 2 to Section 3:**

**Section 2 (Stateless RAG):**
- ❌ No conversation history
- ❌ Each query independent
- ❌ Can't resolve references
- ✅ Retrieves relevant documents

**Section 3 (Memory-Enhanced RAG):**
- ✅ Conversation history (working memory)
- ✅ Multi-turn conversations
- ✅ Reference resolution
- ✅ Persistent user knowledge (long-term memory)
- ✅ Personalization across sessions

### **Next Steps:**

**Section 4** will add **tools** and **agentic workflows** using **LangGraph**, completing your journey from context engineering fundamentals to production-ready AI agents.

---

## 🎉 Congratulations!

You've successfully built a **memory-enhanced RAG system** that:
- Remembers conversations (working memory)
- Accumulates knowledge (long-term memory)
- Resolves references naturally
- Personalizes responses
- Integrates all four context types

**You're now ready for Section 4: Tools & Agentic Workflows!** 🚀




### 🎯 Memory Lifecycle Best Practices

**1. Trust Automatic Extraction**
- Agent Memory Server automatically extracts important facts
- Don't manually store everything in long-term memory
- Let the system decide what's important

**2. Use Appropriate Memory Types**
- Working memory: Current conversation only
- Long-term memory: Facts that should persist

**3. Monitor Memory Growth**
- Long-term memories accumulate over time
- Implement cleanup for outdated information
- Consider archiving old memories

**4. Plan for Expiration**
- Working memory expires after 24 hours
- Important context must be in long-term memory
- Don't rely on working memory for cross-session data

**5. Test Cross-Session Behavior**
- Verify long-term memories are accessible
- Ensure personalization works after TTL expiration
- Test with realistic time gaps

---

## 🎓 Key Takeaways

### **1. Memory Solves the Grounding Problem**

Without memory, agents can't resolve references:
- ❌ "What are **its** prerequisites?" → Agent doesn't know what "its" refers to
- ✅ With working memory → Agent resolves "its" from conversation history

### **2. Two Types of Memory Serve Different Purposes**

**Working Memory (Session-Scoped):**
- Conversation messages from current session
- Enables reference resolution and conversation continuity
- TTL-based (expires after session ends)

**Long-term Memory (Cross-Session):**
- Persistent facts, preferences, goals
- Enables personalization across sessions
- Searchable via semantic vector search

### **3. Memory Completes the Four Context Types**

From Section 1, we learned about four context types. Memory enables two of them:

1. **System Context** (Static) - ✅ Section 2
2. **User Context** (Dynamic, User-Specific) - ✅ Section 2 + Long-term Memory
3. **Conversation Context** (Dynamic, Session-Specific) - ✨ **Working Memory**
4. **Retrieved Context** (Dynamic, Query-Specific) - ✅ Section 2 RAG

### **4. Memory + RAG = Complete Context Engineering**

The integration pattern:
```
1. Load working memory (conversation history)
2. Search long-term memory (user facts)
3. RAG search (relevant documents)
4. Assemble all context types
5. Generate response
6. Save working memory (updated conversation)
```

This gives us **stateful, personalized, context-aware conversations**.

### **5. Agent Memory Server is Production-Ready**

Why use Agent Memory Server instead of simple in-memory storage:
- ✅ **Scalable** - Redis-backed, handles thousands of users
- ✅ **Automatic** - Extracts important facts to long-term storage
- ✅ **Semantic search** - Vector-indexed memory retrieval
- ✅ **Deduplication** - Prevents redundant memories
- ✅ **TTL management** - Automatic expiration of old sessions

### **6. LangChain is Sufficient for Memory + RAG**

We didn't need LangGraph for this section because:
- Simple linear flow (load → search → generate → save)
- No conditional branching or complex state management
- No tool calling required

**LangGraph becomes necessary in Section 4** when we add tools and multi-step workflows.

### **7. Memory Management Best Practices**

**Choose the Right Memory Type:**
- **Semantic** for facts and preferences (most common)
- **Episodic** for time-bound events and timeline
- **Message** for context-rich conversations (use sparingly)

**Understand Memory Lifecycle:**
- **Working memory:** 24-hour TTL, session-scoped
- **Long-term memory:** Indefinite persistence, user-scoped
- **Automatic extraction:** Trust the system to extract important facts

**Benefits of Proper Memory Management:**
- ✅ **Natural conversations** - Users don't repeat themselves
- ✅ **Cross-session personalization** - Knowledge persists over time
- ✅ **Efficient storage** - Automatic deduplication prevents bloat
- ✅ **Semantic search** - Find relevant memories without exact keywords
- ✅ **Scalable** - Redis-backed, production-ready architecture

**Key Principle:** Memory transforms stateless RAG into stateful, personalized, context-aware conversations.

---

## 💪 Practice Exercises

### **Exercise 1: Cross-Session Personalization**

Modify the `memory_enhanced_rag_query` function to:
1. Store user preferences in long-term memory when mentioned
2. Use those preferences in future sessions
3. Test with two different sessions for the same student

**Hint:** Look for phrases like "I prefer...", "I like...", "I want..." and store them as semantic memories.

### **Exercise 2: Memory-Aware Filtering**

Enhance the RAG search to use long-term memories as filters:
1. Search long-term memory for preferences (format, difficulty, schedule)
2. Apply those preferences as filters to `course_manager.search_courses()`
3. Compare results with and without memory-aware filtering

**Hint:** Use the `filters` parameter in `course_manager.search_courses()`.

### **Exercise 3: Conversation Summarization**

Implement a function that summarizes long conversations:
1. When working memory exceeds 10 messages, summarize the conversation
2. Store the summary in long-term memory
3. Clear old messages from working memory (keep only recent 4)
4. Test that reference resolution still works with summarized history

**Hint:** Use the LLM to generate summaries, then store as semantic memories.

### **Exercise 4: Multi-User Memory Management**

Create a simple CLI that:
1. Supports multiple students (different user IDs)
2. Maintains separate working memory per session
3. Maintains separate long-term memory per user
4. Demonstrates cross-session continuity for each user

**Hint:** Use different `session_id` and `user_id` for each student.

### **Exercise 5: Memory Search Quality**

Experiment with long-term memory search:
1. Store 20+ diverse memories for a student
2. Try different search queries
3. Analyze which memories are retrieved
4. Adjust memory text to improve search relevance

**Hint:** More specific memory text leads to better semantic search results.

---

## 📝 Summary

### **What You Learned:**

1. **The Grounding Problem** - Why agents need memory to resolve references
2. **Working Memory** - Session-scoped conversation history for continuity
3. **Long-term Memory** - Cross-session persistent knowledge for personalization
4. **Memory Integration** - Combining memory with Section 2's RAG system
5. **Complete Context Engineering** - All four context types working together
6. **Production Architecture** - Using Agent Memory Server for scalable memory

### **What You Built:**

- ✅ Working memory demo (multi-turn conversations)
- ✅ Long-term memory demo (persistent knowledge)
- ✅ Complete memory-enhanced RAG system
- ✅ Integration of all four context types

### **Key Functions:**

- `memory_enhanced_rag_query()` - Complete memory + RAG pipeline
- `working_memory_demo()` - Demonstrates conversation continuity
- `longterm_memory_demo()` - Demonstrates persistent knowledge
- `complete_demo()` - End-to-end multi-turn conversation

### **Architecture Pattern:**

```
User Query
    ↓
Load Working Memory (conversation history)
    ↓
Search Long-term Memory (user facts)
    ↓
RAG Search (relevant courses)
    ↓
Assemble Context (System + User + Conversation + Retrieved)
    ↓
Generate Response
    ↓
Save Working Memory (updated conversation)
```

### **From Section 2 to Section 3:**

**Section 2 (Stateless RAG):**
- ❌ No conversation history
- ❌ Each query independent
- ❌ Can't resolve references
- ✅ Retrieves relevant documents

**Section 3 (Memory-Enhanced RAG):**
- ✅ Conversation history (working memory)
- ✅ Multi-turn conversations
- ✅ Reference resolution
- ✅ Persistent user knowledge (long-term memory)
- ✅ Personalization across sessions

### **Next Steps:**

**Section 4** will add **tools** and **agentic workflows** using **LangGraph**, completing your journey from context engineering fundamentals to production-ready AI agents.

---

## 🎉 Congratulations!

You've successfully built a **memory-enhanced RAG system** that:
- Remembers conversations (working memory)
- Accumulates knowledge (long-term memory)
- Resolves references naturally
- Personalizes responses
- Integrates all four context types

**You're now ready for Section 4: Tools & Agentic Workflows!** 🚀


